In [2]:
# importing libraries here to better keep track of them

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Reading csv files into memory

injury_history_raw = pd.read_csv("data/injury_history(injury_history).csv", sep = ",", encoding = 'ISO-8859-1')
muscle_imbalance_raw = pd.read_csv("data/injury_history(muscle_imbalance_data).csv", sep = ",", encoding = 'ISO-8859-1')
player_sessions_raw = pd.read_csv("data/injury_history(player_sessions).csv", sep = ",", encoding = 'ISO-8859-1')

In [4]:
print(f"Raw Injury Data: \n {injury_history_raw.head()} \n")
print(f"Raw Muscle Imbalance Data: \n {muscle_imbalance_raw.head()} \n")
print(f"Raw Player Session Data: \n {player_sessions_raw.head()} \n")

Raw Injury Data: 
    Player.ID             Name  Group.Id    Injury Type   Body Part   Side  \
0        101  Jordan Matthews       201  Muscle Strain  Quadriceps  Right   
1        101  Jordan Matthews       201     Tendonitis       Wrist   Left   
2        101  Jordan Matthews       201     Tendonitis    Shoulder  Right   
3        103   Malik Robinson       203         Strain       Groin  Right   
4        103   Malik Robinson       203       Fracture       Wrist   Left   

  Injury Date Severity  Recovery Time (days)  \
0   12/5/2023  Grade 2                    51   
1  10/25/2023      NaN                    11   
2   7/22/2023      NaN                    12   
3   6/28/2023  Grade 1                    20   
4   2/14/2023      NaN                    68   

                                    Additional Notes  
0  Grade 2 quadriceps strain with partial tearing...  
1  De Quervain's tenosynovitis. Swelling and pain...  
2  Rotator cuff tendonitis due to overuse. Anti-i...  
3  Grade 

In [5]:
print(f'Columns present in injury data: \n {injury_history_raw.columns} \n')
print(f'Columns present in muscle imbalance data: \n {muscle_imbalance_raw.columns} \n')
print(f'Columns present in player session data: \n {player_sessions_raw.columns} \n')

Columns present in injury data: 
 Index(['Player.ID', 'Name', 'Group.Id', 'Injury Type', 'Body Part', 'Side',
       'Injury Date', 'Severity', 'Recovery Time (days)', 'Additional Notes'],
      dtype='object') 

Columns present in muscle imbalance data: 
 Index(['Player.ID', 'Session ID', 'Player Name', 'Date Recorded',
       'Hamstring To Quad Ratio', 'Quad Imbalance Percent',
       'HamstringImbalance Percent', 'Calf Imbalance Percent',
       'Groin Imbalance Percent'],
      dtype='object') 

Columns present in player session data: 
 Index(['Name', 'Player.ID', 'Group.Id', 'Group.name', 'League.ID',
       'Session.ID', 'Session_Date', 'Position', 'Distance..mi.',
       'Distance...min..mi.', 'Duration..s.', 'Steps', 'Speed....of.max......',
       'Speed..max....mph.', 'Speed..?ò...mph.', 'Time..s.',
       'Accumulated.Acceleration.Load', 'Anaerobic.Activity..distance...mi.',
       'Jump.Load..J.', 'Heart.Rate..?ò...bpm.', 'Heart.Rate..min....bpm.',
       'Heart.Rate..max..

## Player Session Data column research

#### TRIMP - A way to measure Training Impulse (Aerobic Training Load). It is a product of Training volume (minutes) and Training intensity (beats per minutes). 

TRIMP = Training VOlume * Training Intensity. For example if a workout is 50 minutes and the average heart rate (bpm) is 140 bpm, TRIMP score is calculated out to 7000. TRIMP is affected by a number of factors but we can look for correlation on a correlation matrix as well. 
    - Physical Settings
    - Resting and Maximal Heart Rate
    - Gender (since its mbb there's not going to be any variance with this feature)

TRIMP can be used to compare sessions of different lengths, or to compare high-intensity sessions with longer game data. For example if a session with TRIMP score of 108 might be considered moderate, the TRIMP/min score could indicate an intense session if Training Volume is lower. Training Stress Score (TSS) is a proprietary variant based on a score of 100 for a 1-hour maximum sustained effort. 
Positive correlation between Heart Rate and Effort

---

In [12]:
print(f'Size of injury history data: \n {injury_history_raw.shape}\n')
print(f'Size of muscle imbalance data: \n {muscle_imbalance_raw.shape}\n')
print(f'Size of player session data: \n {player_sessions_raw.shape}\n')

Size of injury history data: 
 (21, 10)

Size of muscle imbalance data: 
 (182, 9)

Size of player session data: 
 (2604, 30)



## Null Values

In [7]:
print(f'Null Values in injury history data: \n {injury_history_raw.isnull().sum()}')
print(f'Null Values in muscle imbalance data: \n {muscle_imbalance_raw.isnull().sum()}')
print(f'Null Values in player session data: \n {player_sessions_raw.isnull().sum()}')

Null Values in injury history data: 
 Player.ID                0
Name                     0
Group.Id                 0
Injury Type              0
Body Part                0
Side                     5
Injury Date              0
Severity                10
Recovery Time (days)     0
Additional Notes         0
dtype: int64
Null Values in muscle imbalance data: 
 Player.ID                     0
Session ID                    0
Player Name                   0
Date Recorded                 0
Hamstring To Quad Ratio       0
Quad Imbalance Percent        0
HamstringImbalance Percent    0
Calf Imbalance Percent        0
Groin Imbalance Percent       0
dtype: int64
Null Values in player session data: 
 Name                                  0
Player.ID                             0
Group.Id                              0
Group.name                            0
League.ID                             0
Session.ID                            0
Session_Date                          0
Position            

Okay so with null values, injury history data is the only dataset with null values. There are 2 columns with null values with a total of 15 null values. Less than 10% of the data. We should delete the data since we are working with health data instead of imputing the values. I think it will introduce bias through assumption. Our train test split is going to be pretty weak since we only will have 167 instances. 

In [22]:
twothird_data = pd.merge(muscle_imbalance_raw, player_sessions_raw, on = 'Player.ID')
twothird_data.head()

,Player.ID,Session ID,Player Name,Date Recorded,Hamstring To Quad Ratio,Quad Imbalance Percent,HamstringImbalance Percent,Calf Imbalance Percent,Groin Imbalance Percent,Name,...,Heart.Rate..min....bpm.,Heart.Rate..max....bpm.,Human.Core.Temperature..?ò....F.,Human.Core.Temperature..max.....F.,TRIMP,Heart.Rate.Recoveries,Jump.Height..max....ft.,Changes.of.Orientation,Exertions,Disk.Usage....
0,112,101,Anthony Lopez,1/1/2023,0.808741,-10.149294,-8.208145,-10.176416,-10.258755,Anthony Lopez,...,74,198,99.47,101.24,261,5,2.31,229,307,58.56
1,112,101,Anthony Lopez,1/1/2023,0.808741,-10.149294,-8.208145,-10.176416,-10.258755,Anthony Lopez,...,62,179,99.56,99.33,270,6,2.44,427,180,44.93
2,112,101,Anthony Lopez,1/1/2023,0.808741,-10.149294,-8.208145,-10.176416,-10.258755,Anthony Lopez,...,78,172,100.06,102.31,149,4,3.04,383,440,15.32
3,112,101,Anthony Lopez,1/1/2023,0.808741,-10.149294,-8.208145,-10.176416,-10.258755,Anthony Lopez,...,64,186,100.45,101.10,180,10,3.17,462,450,21.46
4,112,101,Anthony Lopez,1/1/2023,0.808741,-10.149294,-8.208145,-10.176416,-10.258755,Anthony Lopez,...,62,146,98.73,100.91,152,4,1.28,118,416,20.51


In [23]:
obt = pd.merge(twothird_data, injury_history_raw, on = 'Player.ID')
obt.head()

,Player.ID,Session ID,Player Name,Date Recorded,Hamstring To Quad Ratio,Quad Imbalance Percent,HamstringImbalance Percent,Calf Imbalance Percent,Groin Imbalance Percent,Name_x,...,Disk.Usage....,Name_y,Group.Id_y,Injury Type,Body Part,Side,Injury Date,Severity,Recovery Time (days),Additional Notes
0,112,101,Anthony Lopez,1/1/2023,0.808741,-10.149294,-8.208145,-10.176416,-10.258755,Anthony Lopez,...,58.56,Anthony Lopez,212,Strain,Knee,Left,1/26/2023,Grade 1,28,Strain of the posterior cruciate ligament (PCL...
1,112,101,Anthony Lopez,1/1/2023,0.808741,-10.149294,-8.208145,-10.176416,-10.258755,Anthony Lopez,...,58.56,Anthony Lopez,212,Muscle Strain,Quadriceps,Left,11/18/2023,Grade 1,12,Grade 1 quadriceps strain. Minor tear in the m...
2,112,101,Anthony Lopez,1/1/2023,0.808741,-10.149294,-8.208145,-10.176416,-10.258755,Anthony Lopez,...,58.56,Anthony Lopez,212,Concussion,Head,NaN,7/15/2023,Grade 1,11,Grade 1 concussion following a collision. Symp...
3,112,101,Anthony Lopez,1/1/2023,0.808741,-10.149294,-8.208145,-10.176416,-10.258755,Anthony Lopez,...,44.93,Anthony Lopez,212,Strain,Knee,Left,1/26/2023,Grade 1,28,Strain of the posterior cruciate ligament (PCL...
4,112,101,Anthony Lopez,1/1/2023,0.808741,-10.149294,-8.208145,-10.176416,-10.258755,Anthony Lopez,...,44.93,Anthony Lopez,212,Muscle Strain,Quadriceps,Left,11/18/2023,Grade 1,12,Grade 1 quadriceps strain. Minor tear in the m...


In [19]:
obt.isnull().sum()

Player.ID                                 0
Session ID                                0
Player Name                               0
Date Recorded                             0
Hamstring To Quad Ratio                   0
Quad Imbalance Percent                    0
HamstringImbalance Percent                0
Calf Imbalance Percent                    0
Groin Imbalance Percent                   0
Name_x                                    0
Group.Id_x                                0
Group.name                                0
League.ID                                 0
Session.ID                                0
Session_Date                              0
Position                                  0
Distance..mi.                             0
Distance...min..mi.                       0
Duration..s.                              0
Steps                                     0
Speed....of.max......                     0
Speed..max....mph.                        0
Speed..?ò...mph.                

In [28]:
obt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44837 entries, 0 to 44836
Data columns (total 47 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Player.ID                           44837 non-null  int64  
 1   Session ID                          44837 non-null  int64  
 2   Player Name                         44837 non-null  object 
 3   Date Recorded                       44837 non-null  object 
 4   Hamstring To Quad Ratio             44837 non-null  float64
 5   Quad Imbalance Percent              44837 non-null  float64
 6   HamstringImbalance Percent          44837 non-null  float64
 7   Calf Imbalance Percent              44837 non-null  float64
 8   Groin Imbalance Percent             44837 non-null  float64
 9   Name_x                              44837 non-null  object 
 10  Group.Id_x                          44837 non-null  int64  
 11  Group.name                          44837